# Data Pre-Processing

### Import and present data

In [37]:
import pandas as pd
import os
from io import StringIO
import re

# Function to read a CSV file, skip comment lines starting with '%', and stopping at lines starting with '#'
def read_csv_with_comments(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Filter out comment lines and stop at the ending line
    data_lines = []
    for line in lines:
        if line.startswith('#'):
            break
        if not line.startswith('%'):
            data_lines.append(line.strip())

    # Check if data_lines is empty
    if len(data_lines) == 0:
        return None  # Return None if the file has no data

    return data_lines


# Select the data folder and import all the data files
data_folder = 'Data/ROADEF 2009/A1_6088570/A01_6088570'

aircraft_file = data_folder + '/aircraft.csv'
airports_file = data_folder + '/airports.csv'
alt_aircraft_file = data_folder + '/alt_aircraft.csv'
alt_airports_file = data_folder + '/alt_airports.csv'
alt_flights_file = data_folder + '/alt_flights.csv'
config_file = data_folder + '/config.csv'
dist_file = data_folder + '/dist.csv'
flights_file = data_folder + '/flights.csv'
itineraries_file = data_folder + '/itineraries.csv'
positions_file = data_folder + '/position.csv'
rotations_file = data_folder + '/rotations.csv'


# Printing data in this instance

for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(data_folder, file)
        data_lines = read_csv_with_comments(file_path)
        if data_lines is not None:
            print(file)
            print(data_lines[:5])
            print()
        else:
            # print the file name if it has no data
            print(file)
            print('No data')
            print()


aircraft.csv
['A318#1 A318 Airbus 0/0/123 450 1800.0 30 30 CFE NULL', 'A318#2 A318 Airbus 0/0/123 450 1800.0 30 30 ETZ NULL', 'A318#3 A318 Airbus 0/0/123 450 1800.0 30 30 NTE NULL', 'A318#4 A318 Airbus 0/0/123 450 1800.0 30 30 NCE NULL', 'A318#5 A318 Airbus 0/0/123 450 1800.0 30 30 ORY NULL']

alt_flights.csv
['145 07/01/06 9', '146 07/01/06 14', '152 07/01/06 6', '1378 07/01/06 6', '2518 07/01/06 38']

alt_airports.csv
No data

alt_aircraft.csv
No data

config.csv
['07/01/06 12:00 08/01/06 04:00', 'F D 1.25 F C 1.25 F I 1.25 B D 0.8 B C 0.85 B I 0.9 E D 0.05 E C 0.15 E I 0.25', 'F D 2500.0 F C 2750.0 F I 3000.0 B D 1500.0 B C 1750.0 B I 2000.0 E D 250.0 E C 600.0 E I 1000.0', 'F D 7500.0 F C 8250.0 F I 9000.0 B D 4500.0 B C 5250.0 B I 6000.0 E D 750.0 E C 1500.0 E I 3000.0', 'F B D 150.0 F B C 400.0 F B I 750.0 F E D 200.0 F E C 500.0 F E I 1500.0 B E D 150.0 B E C 400.0 B E I 750.0']

rotations.csv
['4344 07/01/06 CRJ100#1', '4343 07/01/06 CRJ100#1', '4502 07/01/06 CRJ100#1', '4501 0

## Each file has its own format

### Config file

In [38]:
# Function to convert config file lines to a dictionary with corresponding values
def parse_config(data_lines):
    config_dict = {}
    config_dict['RecoveryPeriod'] = {
        'StartDate': data_lines[0].split()[0],
        'StartTime': data_lines[0].split()[1],
        'EndDate': data_lines[0].split()[2],
        'EndTime': data_lines[0].split()[3]
    }

    def parse_costs(line):
        parts = re.split(r'\s+', line)
        costs = []
        for i in range(0, len(parts), 3):
            costs.append({'Cabin': parts[i], 'Type': parts[i+1], 'Cost': float(parts[i+2])})
        return costs

    config_dict['DelayCosts'] = parse_costs(data_lines[1])
    config_dict['CancellationCostsOutbound'] = parse_costs(data_lines[2])
    config_dict['CancellationCostsInbound'] = parse_costs(data_lines[3])

    def parse_downgrading_costs(line):
        parts = re.split(r'\s+', line)
        costs = []
        for i in range(0, len(parts), 4):
            costs.append({'FromCabin': parts[i], 'ToCabin': parts[i+1], 'Type': parts[i+2], 'Cost': float(parts[i+3])})
        return costs

    config_dict['DowngradingCosts'] = parse_downgrading_costs(data_lines[4])
    config_dict['PenaltyCosts'] = [float(x) for x in re.split(r'\s+', data_lines[5])]
    config_dict['Weights'] = [float(x) for x in re.split(r'\s+', data_lines[6])]
    return config_dict

# Read the config file and parse the data
config_lines = read_csv_with_comments(config_file)
if config_lines:
    config_dict = parse_config(config_lines)
    print(config_dict)


{'RecoveryPeriod': {'StartDate': '07/01/06', 'StartTime': '12:00', 'EndDate': '08/01/06', 'EndTime': '04:00'}, 'DelayCosts': [{'Cabin': 'F', 'Type': 'D', 'Cost': 1.25}, {'Cabin': 'F', 'Type': 'C', 'Cost': 1.25}, {'Cabin': 'F', 'Type': 'I', 'Cost': 1.25}, {'Cabin': 'B', 'Type': 'D', 'Cost': 0.8}, {'Cabin': 'B', 'Type': 'C', 'Cost': 0.85}, {'Cabin': 'B', 'Type': 'I', 'Cost': 0.9}, {'Cabin': 'E', 'Type': 'D', 'Cost': 0.05}, {'Cabin': 'E', 'Type': 'C', 'Cost': 0.15}, {'Cabin': 'E', 'Type': 'I', 'Cost': 0.25}], 'CancellationCostsOutbound': [{'Cabin': 'F', 'Type': 'D', 'Cost': 2500.0}, {'Cabin': 'F', 'Type': 'C', 'Cost': 2750.0}, {'Cabin': 'F', 'Type': 'I', 'Cost': 3000.0}, {'Cabin': 'B', 'Type': 'D', 'Cost': 1500.0}, {'Cabin': 'B', 'Type': 'C', 'Cost': 1750.0}, {'Cabin': 'B', 'Type': 'I', 'Cost': 2000.0}, {'Cabin': 'E', 'Type': 'D', 'Cost': 250.0}, {'Cabin': 'E', 'Type': 'C', 'Cost': 600.0}, {'Cabin': 'E', 'Type': 'I', 'Cost': 1000.0}], 'CancellationCostsInbound': [{'Cabin': 'F', 'Type': 'D

### Airports file

In [39]:
def parse_airports(data_lines):
    airports_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        airport = parts[0]
        capacities = []
        for i in range(1, len(parts), 4):
            capacities.append({
                'Dep/h': int(parts[i]),
                'Arr/h': int(parts[i+1]),
                'StartTime': parts[i+2],
                'EndTime': parts[i+3]
            })
        airports_dict[airport] = capacities
    return airports_dict


airports_lines = read_csv_with_comments(airports_file)
if airports_lines:
    airports_dict = parse_airports(airports_lines)
    print(airports_dict)




{'AJA': [{'Dep/h': 0, 'Arr/h': 0, 'StartTime': '00:00', 'EndTime': '05:00'}, {'Dep/h': 1, 'Arr/h': 1, 'StartTime': '05:00', 'EndTime': '14:00'}, {'Dep/h': 2, 'Arr/h': 1, 'StartTime': '14:00', 'EndTime': '16:00'}, {'Dep/h': 1, 'Arr/h': 1, 'StartTime': '16:00', 'EndTime': '18:00'}, {'Dep/h': 1, 'Arr/h': 2, 'StartTime': '18:00', 'EndTime': '20:00'}, {'Dep/h': 1, 'Arr/h': 1, 'StartTime': '20:00', 'EndTime': '00:00'}], 'AMS': [{'Dep/h': 1, 'Arr/h': 1, 'StartTime': '00:00', 'EndTime': '07:00'}, {'Dep/h': 4, 'Arr/h': 2, 'StartTime': '07:00', 'EndTime': '09:00'}, {'Dep/h': 1, 'Arr/h': 1, 'StartTime': '09:00', 'EndTime': '12:00'}, {'Dep/h': 2, 'Arr/h': 3, 'StartTime': '12:00', 'EndTime': '15:00'}, {'Dep/h': 1, 'Arr/h': 1, 'StartTime': '15:00', 'EndTime': '17:00'}, {'Dep/h': 3, 'Arr/h': 3, 'StartTime': '17:00', 'EndTime': '19:00'}, {'Dep/h': 1, 'Arr/h': 1, 'StartTime': '19:00', 'EndTime': '00:00'}], 'AVN': [{'Dep/h': 0, 'Arr/h': 0, 'StartTime': '00:00', 'EndTime': '05:00'}, {'Dep/h': 1, 'Arr/h':

### Distance file

In [40]:

def parse_dist(data_lines):
    dist_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        origin = parts[0]
        destination = parts[1]
        dist = int(parts[2])
        type_ = parts[3]
        dist_dict[(origin, destination)] = {'Dist': dist, 'Type': type_}
    return dist_dict



dist_lines = read_csv_with_comments(dist_file)
if dist_lines:
    dist_dict = parse_dist(dist_lines)
    print(dist_dict)


{('AJA', 'AMS'): {'Dist': 120, 'Type': 'C'}, ('AJA', 'AVN'): {'Dist': 60, 'Type': 'D'}, ('AJA', 'BES'): {'Dist': 125, 'Type': 'D'}, ('AJA', 'BIA'): {'Dist': 40, 'Type': 'D'}, ('AJA', 'BIQ'): {'Dist': 95, 'Type': 'D'}, ('AJA', 'BOD'): {'Dist': 95, 'Type': 'D'}, ('AJA', 'CDG'): {'Dist': 100, 'Type': 'D'}, ('AJA', 'CFE'): {'Dist': 80, 'Type': 'D'}, ('AJA', 'CFR'): {'Dist': 115, 'Type': 'D'}, ('AJA', 'CLY'): {'Dist': 35, 'Type': 'D'}, ('AJA', 'ETZ'): {'Dist': 95, 'Type': 'D'}, ('AJA', 'FSC'): {'Dist': 35, 'Type': 'D'}, ('AJA', 'LDE'): {'Dist': 85, 'Type': 'D'}, ('AJA', 'LEH'): {'Dist': 115, 'Type': 'D'}, ('AJA', 'LGW'): {'Dist': 125, 'Type': 'D'}, ('AJA', 'LIG'): {'Dist': 90, 'Type': 'D'}, ('AJA', 'LIL'): {'Dist': 110, 'Type': 'D'}, ('AJA', 'LRT'): {'Dist': 120, 'Type': 'D'}, ('AJA', 'LYS'): {'Dist': 95, 'Type': 'D'}, ('AJA', 'MLH'): {'Dist': 80, 'Type': 'D'}, ('AJA', 'MPL'): {'Dist': 65, 'Type': 'D'}, ('AJA', 'MRS'): {'Dist': 55, 'Type': 'D'}, ('AJA', 'NCE'): {'Dist': 55, 'Type': 'D'}, ('

### Flights file

In [41]:



def parse_flights(data_lines):
    flights_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        flight = int(parts[0])
        flights_dict[flight] = {
            'Orig': parts[1],
            'Dest': parts[2],
            'DepTime': parts[3],
            'ArrTime': parts[4],
            'PrevFlight': int(parts[5])
        }
    return flights_dict


flights_lines = read_csv_with_comments(flights_file)
if flights_lines:
    flights_dict = parse_flights(flights_lines)
    print(flights_dict)



{1: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '00:00', 'ArrTime': '00:30', 'PrevFlight': 0}, 2: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '00:20', 'ArrTime': '00:50', 'PrevFlight': 0}, 3: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '00:40', 'ArrTime': '01:10', 'PrevFlight': 0}, 4: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '01:00', 'ArrTime': '01:30', 'PrevFlight': 0}, 5: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '01:20', 'ArrTime': '01:50', 'PrevFlight': 0}, 6: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '01:40', 'ArrTime': '02:10', 'PrevFlight': 0}, 7: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '02:00', 'ArrTime': '02:30', 'PrevFlight': 0}, 8: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '02:20', 'ArrTime': '02:50', 'PrevFlight': 0}, 9: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '02:40', 'ArrTime': '03:10', 'PrevFlight': 0}, 10: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '03:00', 'ArrTime': '03:30', 'PrevFlight': 0}, 11: {'Orig': 'CDG', 'Dest': 'ORY', 'DepTime': '03:20', 'ArrTime': '03:50', 'Pr

### Aircraft file

In [42]:


def parse_aircraft(data_lines):
    aircraft_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        aircraft = parts[0]
        aircraft_dict[aircraft] = {
            'Model': parts[1],
            'Family': parts[2],
            'Config': parts[3],
            'Dist': int(parts[4]),
            'Cost/h': float(parts[5]),
            'TurnRound': int(parts[6]),
            'Transit': int(parts[7]),
            'Orig': parts[8],
            'Maint': parts[9] if len(parts) > 9 else None
        }
    return aircraft_dict

aircraft_lines = read_csv_with_comments(aircraft_file)
if aircraft_lines:
    aircraft_dict = parse_aircraft(aircraft_lines)
    print(aircraft_dict)

    


{'A318#1': {'Model': 'A318', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 450, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': 'CFE', 'Maint': 'NULL'}, 'A318#2': {'Model': 'A318', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 450, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': 'ETZ', 'Maint': 'NULL'}, 'A318#3': {'Model': 'A318', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 450, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': 'NTE', 'Maint': 'NULL'}, 'A318#4': {'Model': 'A318', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 450, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': 'NCE', 'Maint': 'NULL'}, 'A318#5': {'Model': 'A318', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 450, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': 'ORY', 'Maint': 'NULL'}, 'A318#6': {'Model': 'A318', 'Family': 'Airbus', 'Config': '0/0/123', 'Dist': 450, 'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': 'LIG', 'Maint': 'NULL'}, 'A318#7':

### Rotations file

In [43]:

def parse_rotations(data_lines):
    rotations_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        flight = int(parts[0])
        rotations_dict[flight] = {
            'DepDate': parts[1],
            'Aircraft': parts[2]
        }
    return rotations_dict

rotations_lines = read_csv_with_comments(rotations_file)
if rotations_lines:
    rotations_dict = parse_rotations(rotations_lines)
    print(rotations_dict)



{4344: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4343: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4502: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4501: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4352: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4347: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4358: {'DepDate': '07/01/06', 'Aircraft': 'CRJ100#1'}, 4334: {'DepDate': '07/01/06', 'Aircraft': 'ERJ145#1'}, 4333: {'DepDate': '07/01/06', 'Aircraft': 'ERJ145#1'}, 4336: {'DepDate': '07/01/06', 'Aircraft': 'ERJ145#1'}, 4337: {'DepDate': '07/01/06', 'Aircraft': 'ERJ145#1'}, 4224: {'DepDate': '07/01/06', 'Aircraft': 'A320#1'}, 4225: {'DepDate': '07/01/06', 'Aircraft': 'A320#1'}, 4228: {'DepDate': '07/01/06', 'Aircraft': 'A320#1'}, 4239: {'DepDate': '07/01/06', 'Aircraft': 'A320#1'}, 4238: {'DepDate': '07/01/06', 'Aircraft': 'A320#1'}, 4237: {'DepDate': '07/01/06', 'Aircraft': 'A320#1'}, 1364: {'DepDate': '07/01/06', 'Aircraft': 'A320#2'}, 1363: {'DepDate': '07/0

### Itineraries file

In [44]:

def parse_itineraries(data_lines):
    itineraries_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        ident = int(parts[0])
        itineraries_dict[ident] = {
            'Type': parts[1],
            'Price': float(parts[2]),
            'Count': int(parts[3]),
            'Flights': parts[4:]
        }
    return itineraries_dict

itineraries_lines = read_csv_with_comments(itineraries_file)
if itineraries_lines:
    itineraries_dict = parse_itineraries(itineraries_lines)
    print(itineraries_dict)



{1: {'Type': 'A', 'Price': 200.0, 'Count': 10, 'Flights': ['4344', '07/01/06', 'E']}, 2: {'Type': 'A', 'Price': 287.5, 'Count': 3, 'Flights': ['4344', '07/01/06', 'E', '4333', '07/01/06', 'E']}, 3: {'Type': 'R', 'Price': 200.0, 'Count': 16, 'Flights': ['4344', '07/01/06', 'E']}, 4: {'Type': 'R', 'Price': 287.5, 'Count': 2, 'Flights': ['4344', '07/01/06', 'E', '4333', '07/01/06', 'E']}, 5: {'Type': 'A', 'Price': 287.5, 'Count': 1, 'Flights': ['4344', '07/01/06', 'E', '4347', '07/01/06', 'E']}, 6: {'Type': 'A', 'Price': 287.5, 'Count': 1, 'Flights': ['4344', '07/01/06', 'E', '4351', '07/01/06', 'E']}, 7: {'Type': 'A', 'Price': 500.0, 'Count': 5, 'Flights': ['4344', '07/01/06', 'E', '4225', '07/01/06', 'E', '4238', '07/01/06', 'E', '4347', '07/01/06', 'E']}, 8: {'Type': 'A', 'Price': 187.5, 'Count': 10, 'Flights': ['4343', '07/01/06', 'E']}, 9: {'Type': 'R', 'Price': 187.5, 'Count': 6, 'Flights': ['4343', '07/01/06', 'E']}, 10: {'Type': 'A', 'Price': 287.5, 'Count': 1, 'Flights': ['4343',

### Positions file

In [45]:

def parse_positions(data_lines):
    positions_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        airport = parts[0]
        model = parts[1]
        config = parts[2]
        count = int(parts[3])
        if airport not in positions_dict:
            positions_dict[airport] = []
        positions_dict[airport].append({'Model': model, 'Config': config, 'Count': count})
    return positions_dict


positions_lines = read_csv_with_comments(positions_file)
if positions_lines:
    positions_dict = parse_positions(positions_lines)
    print(positions_dict)


    


{'ORY': [{'Model': 'CRJ100', 'Config': '0/0/50', 'Count': 2}], 'LEH': [{'Model': 'ERJ135', 'Config': '0/0/37', 'Count': 1}], 'CFR': [{'Model': 'ERJ145', 'Config': '0/0/50', 'Count': 1}], 'LIL': [{'Model': 'F100', 'Config': '0/0/100', 'Count': 1}], 'NCE': [{'Model': 'A318', 'Config': '0/0/123', 'Count': 2}], 'BIQ': [{'Model': 'A320', 'Config': '0/0/172', 'Count': 1}], 'LYS': [{'Model': 'A321', 'Config': '0/0/206', 'Count': 1}], 'AVN': [{'Model': 'ERJ145', 'Config': '0/0/50', 'Count': 1}], 'BES': [{'Model': 'A320', 'Config': '0/0/172', 'Count': 1}], 'AJA': [{'Model': 'A320', 'Config': '0/0/172', 'Count': 1}], 'BIA': [{'Model': 'A320', 'Config': '0/0/172', 'Count': 1}], 'CFE': [{'Model': 'A318', 'Config': '0/0/123', 'Count': 1}], 'ETZ': [{'Model': 'A318', 'Config': '0/0/123', 'Count': 1}], 'LDE': [{'Model': 'ERJ145', 'Config': '0/0/50', 'Count': 1}], 'LIG': [{'Model': 'BAE300', 'Config': '0/0/100', 'Count': 1}], 'PUF': [{'Model': 'A319', 'Config': '0/0/142', 'Count': 1}], 'BOD': [{'Model'

### Flights disruptions file

In [46]:

def parse_alt_flights(data_lines):
    alt_flights_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        flight = int(parts[0])
        alt_flights_dict[flight] = {
            'DepDate': parts[1],
            'Delay': int(parts[2])
        }
    return alt_flights_dict

alt_flights_lines = read_csv_with_comments(alt_flights_file)
if alt_flights_lines:
    alt_flights_dict = parse_alt_flights(alt_flights_lines)
    print(alt_flights_dict)




{145: {'DepDate': '07/01/06', 'Delay': 9}, 146: {'DepDate': '07/01/06', 'Delay': 14}, 152: {'DepDate': '07/01/06', 'Delay': 6}, 1378: {'DepDate': '07/01/06', 'Delay': 6}, 2518: {'DepDate': '07/01/06', 'Delay': 38}, 2521: {'DepDate': '07/01/06', 'Delay': 19}, 2522: {'DepDate': '07/01/06', 'Delay': 24}, 2526: {'DepDate': '07/01/06', 'Delay': 27}, 2581: {'DepDate': '07/01/06', 'Delay': 43}, 2593: {'DepDate': '07/01/06', 'Delay': 20}, 2597: {'DepDate': '07/01/06', 'Delay': 4}, 2598: {'DepDate': '07/01/06', 'Delay': 11}, 2605: {'DepDate': '07/01/06', 'Delay': 18}, 2606: {'DepDate': '07/01/06', 'Delay': 18}, 2647: {'DepDate': '07/01/06', 'Delay': 75}, 2657: {'DepDate': '07/01/06', 'Delay': 19}, 2661: {'DepDate': '07/01/06', 'Delay': 90}, 2662: {'DepDate': '07/01/06', 'Delay': 84}, 2868: {'DepDate': '07/01/06', 'Delay': 19}, 2877: {'DepDate': '07/01/06', 'Delay': 17}, 2883: {'DepDate': '07/01/06', 'Delay': 8}, 2888: {'DepDate': '07/01/06', 'Delay': 27}, 2969: {'DepDate': '07/01/06', 'Delay': 

### Aircraft disruptions file

In [47]:


def parse_alt_aircraft(data_lines):
    alt_aircraft_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        aircraft = parts[0]
        alt_aircraft_dict[aircraft] = {
            'StartDate': parts[1],
            'StartTime': parts[2],
            'EndDate': parts[3],
            'EndTime': parts[4]
        }
    return alt_aircraft_dict

alt_aircraft_lines = read_csv_with_comments(alt_aircraft_file)
if alt_aircraft_lines:
    alt_aircraft_dict = parse_alt_aircraft(alt_aircraft_lines)
    print(alt_aircraft_dict)



### Airport disruptions file

In [48]:


def parse_alt_airports(data_lines):
    alt_airports_dict = {}
    for line in data_lines:
        parts = re.split(r'\s+', line)
        airport = parts[0]
        alt_airports_dict[airport] = {
            'StartDate': parts[1],
            'StartTime': parts[2],
            'EndDate': parts[3],
            'EndTime': parts[4],
            'Dep/h': int(parts[5]),
            'Arr/h': int(parts[6])
        }
    return alt_airports_dict


alt_airports_lines = read_csv_with_comments(alt_airports_file)
if alt_airports_lines:
    alt_airports_dict = parse_alt_airports(alt_airports_lines)
    print(alt_airports_dict)
